In [14]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0" #! specify gpu here

import matplotlib
import matplotlib.pyplot as plt

# import seaborn as sns
import json
import warnings
from pathlib import Path
import random
import base64
from io import BytesIO
import cv2
# import vision_pipeline.obb
# import imagesize
# from scipy import ndimage
import natsort
from PIL import Image as PILImage
# from PIL import ImageDraw, ImageFilter
import numpy as np
from tqdm import tqdm
# from shapely.geometry import Polygon
from rich import print
# from types import SimpleNamespace
import pickle
import imutils

# ros package
from context_action_framework.types import Detection, Label, Module, Camera, detections_to_ros, detections_to_py
from sensor_msgs.msg import Image, CameraInfo # CameraInfo needed for pickle

from context_action_framework.srv import VisionDetection, VisionDetectionResponse, VisionDetectionRequest, ProcessImg, ProcessImgResponse

from cv_bridge import CvBridge
import rospy



# # local imports
# from vision_pipeline.helpers import Struct, make_valid_poly, img_to_camera_coords
# from action_predictor.graph_relations import GraphRelations, exists_detection, compute_iou
# from vision_pipeline.work_surface_detection_opencv import WorkSurfaceDetection
# from vision_pipeline.object_detection_model import ObjectDetectionModel
# from vision_pipeline.object_detection import ObjectDetection
# from vision_pipeline.object_reid import ObjectReId

# from vision_pipeline.config import load_config

# from vision_pipeline.object_reid_superglue import ObjectReIdSuperGlue

rospy.init_node("test_node")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:

# TODO: call a service to get info about image

img_path = os.path.expanduser("~/datasets2/reconcycle/2023-08-01_basler_hca_backs/0001.jpg")


img = cv2.imread(img_path)
img = imutils.resize(img, width=1450, height=1450)


# display(PILImage.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))

In [16]:
def call_process_img(img):
    timeout = 3 # 2 second timeout
    rospy.wait_for_service('vision/basler/process_img', timeout)

    imgmsg = CvBridge().cv2_to_imgmsg(img, encoding="bgr8")
    try:
        process_img = rospy.ServiceProxy('vision/basler/process_img', ProcessImg)
        response = process_img(imgmsg)
        detections = detections_to_py(response.detections)
        labelled_img = CvBridge().imgmsg_to_cv2(response.labelled_image, desired_encoding='passthrough')
        
        return response.success, detections, labelled_img
        # return response
    except rospy.ServiceException as e:
        print("Service call failed: %s"%e)

success, detections, labelled_img = call_process_img(img)

display(PILImage.fromarray(cv2.cvtColor(labelled_img, cv2.COLOR_BGR2RGB)))

print("detections", detections)

In [ ]:


def call_get_detection():
    timeout = 3 # 2 second timeout
    service_path = 'vision/basler/get_detection'
    rospy.wait_for_service(service_path, timeout)

    # imgmsg = CvBridge().cv2_to_imgmsg(img, encoding="bgr8")
    try:
        get_detection = rospy.ServiceProxy(service_path, VisionDetection)
        response = get_detection(camera=Camera.basler.value, gap_detection=False)
        return response
    except rospy.ServiceException as e:
        print("Service call failed: %s"%e)

response = call_get_detection()
print("response", response)

response success: False
vision_details: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: ''
  camera_acquisition_stamp: 
    secs: 0
    nsecs:         0
  camera: 0
  gap_detection: False
  detections: []
  gaps: []
image: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: ''
  height: 0
  width: 0
  encoding: ''
  is_bigendian: 0
  step: 0
  data: []